In [17]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from sqlalchemy import create_engine
import time

# 데이터베이스 엔진 설정
engine = create_engine('sqlite:///news_data.db')  # SQLite 데이터베이스 파일 생성


# 웹드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.bigkinds.or.kr/v2/news/recentNews.do')
time.sleep(5)  # 페이지 로드를 기다림

# 언론사 필터 버튼 클릭
press_filter_button = driver.find_element(By.CSS_SELECTOR, 'button.btn-press.btn-toggle')
press_filter_button.click()

# '전국일간지' 체크박스 라벨 대기 및 클릭
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'category_provider_group')))
national_press_label = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for="전국일간지"]'))
)

# JavaScript를 사용하여 라벨 클릭 강제 실행
driver.execute_script("arguments[0].click();", national_press_label)

# 필터 패널 닫기
close_filter_button = driver.find_element(By.CSS_SELECTOR, 'button.close-filter-btn')
close_filter_button.click()

# 변경 사항이 적용되도록 10초간 대기
time.sleep(10)

# 데이터를 저장할 리스트 생성
news_data = []

current_page = 1
max_page = 1  # 원하는 페이지 수 지정

while current_page <= max_page:
    news_blocks = driver.find_elements(By.CSS_SELECTOR, 'div.news-inner')
    for block in news_blocks:
        attempts = 0
        while attempts < 3:
            try:
                # 제목과 출판사 정보 추출
                title = block.find_element(By.CSS_SELECTOR, 'strong.title').text
                publisher = block.find_element(By.CSS_SELECTOR, 'div.info a').text
                news_url = block.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

                # a 태그 클릭
                block.find_element(By.CSS_SELECTOR, 'a').click()
                
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.news-view-body')))
                
                # 날짜, 리포터 이름, 내용 추출
                date_and_reporter = driver.find_elements(By.CSS_SELECTOR, 'ul.info > li')
                news_date = date_and_reporter[0].text if len(date_and_reporter) > 0 else '날짜 정보 없음'
                reporter_name = date_and_reporter[1].text if len(date_and_reporter) > 1 else '기자 정보 없음'
                news_content = driver.find_element(By.CSS_SELECTOR, 'div.news-view-body').text

                # 이미지 추출
                try:
                    image_element = driver.find_element(By.CSS_SELECTOR, 'div.news-view-body img')
                    image_src = image_element.get_attribute('src')
                except NoSuchElementException:
                    image_src = '이미지 없음'

                # 데이터 리스트에 추가
                news_data.append({
                    'Date': news_date,
                    'Title': title,
                    'Press': publisher,
                    'Author': reporter_name,
                    'Content': news_content,
                    'Image': image_src,
                    'URL': news_url,
                })

                # 모달 닫기
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.modal-footer > button'))
                ).click()
                time.sleep(2)
                break
            except Exception as e:
                print(f"Error collecting data: {str(e)}. Retrying...") 
                attempts += 1

    if current_page < max_page:
        next_page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, f'a.page-link[data-page="{current_page + 1}"]'))
        )
        next_page_button.click()
        time.sleep(5)
    current_page += 1

driver.quit()

df = pd.DataFrame(news_data)
# 데이터베이스에 저장
df.to_sql('news', con=engine, index=False, if_exists='append')  # 'news' 테이블에 데이터 추가

print("데이터가 데이터베이스에 저장되었습니다.")


Error collecting data: Message: element click intercepted: Element <a href="#modal-news1" data-toggle="modal" class="thumb news-detail" data-newsid="01100751.20241018144106001" style="background-image: url('/assets/v3/img/content/img-noImg.png');" title="팝업창이 열림"></a> is not clickable at point (492, 430). Other element would receive the click: <div class="news-view-body">...</div>
  (Session info: chrome=130.0.6723.59)
Stacktrace:
0   chromedriver                        0x00000001028d39d4 cxxbridge1$str$ptr + 3647524
1   chromedriver                        0x00000001028cc234 cxxbridge1$str$ptr + 3616900
2   chromedriver                        0x000000010233810c cxxbridge1$string$len + 88416
3   chromedriver                        0x000000010237fa74 cxxbridge1$string$len + 381640
4   chromedriver                        0x000000010237e0dc cxxbridge1$string$len + 375088
5   chromedriver                        0x000000010237c320 cxxbridge1$string$len + 367476
6   chromedriver              

/var/folders/40/58yhrb9d7q32wcxf8vbspv8h0000gn/T/ipykernel_1142/440395431.py:109: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('news', con=engine, index=False, if_exists='append')  # 'news' 테이블에 데이터 추가


AttributeError: 'Engine' object has no attribute 'cursor'

In [51]:
df = pd.read_csv('news_data.csv')

df.head()

,Date,Title,Press,Author,Content,Image,URL
0,2024-10-17,[],NaN,김유성,"박 전 의원, SNS 통해 ""오히려 신뢰 보여주고 응원""; ""삼성이 달라지면 세계 ...",https://www.bigkinds.or.kr/resources/images/04...,https://www.bigkinds.or.kr/v2/news/recentNews....
1,2024-10-17,[],NaN,최홍은 기자,윤석열 대통령은 오는 24일 용산 대통령실에서 안제이 두다 폴란드 대통령과 정상회...,이미지 없음,https://www.bigkinds.or.kr/v2/news/recentNews....
2,2024-10-17,[],NaN,박세린 기자,김철모 시인이 생애 여덟 번째 시집 '채석강에서 가을을 낚다'를 출간했다.\n\n이...,이미지 없음,https://www.bigkinds.or.kr/v2/news/recentNews....
3,2024-10-17,[],NaN,박세린 기자,누벨백미술관에서 16일부터 23일까지 제1회 황선의 개인전을 진행한다.\n\n이번 ...,이미지 없음,https://www.bigkinds.or.kr/v2/news/recentNews....
4,2024-10-17,[],NaN,박세린 기자,세계서예전북비엔날레 조직위원회(이하 조직위)는 18일부터 27일까지 한국소리문화의전...,이미지 없음,https://www.bigkinds.or.kr/v2/news/recentNews....
